In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from unidecode import unidecode

# pd.set_option('display.max_columns', None)

## Prétraitement des données

### Premier tour

In [ ]:
t1_dep = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/bf616688-8478-46fa-8abe-cd1cc1a53b83')

t1_dep.drop(t1_dep.columns[28:52], axis=1, inplace=True)
t1_dep.drop(t1_dep.columns[[3, 5, 6, 7, 8, 10, 11, 13, 14]], axis=1, inplace=True)

t1_dep.columns = ['Code Département','Département', 'Inscrits', 'Taux Abstention', 'Taux Blanc', 'Taux Nul', 'Taux Exprimé',
                  'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LE PEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']

In [ ]:
t1_dep.head()

### Second tour

In [ ]:
t2_dep = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/c7b72e78-5e78-4976-995f-30533f0a35d3')

t2_dep.drop(t2_dep.columns[18:52], axis=1, inplace=True)
t2_dep.drop(t2_dep.columns[[3, 5, 6, 7, 8, 10, 11, 13, 14]], axis=1, inplace=True)

t2_dep.columns = ['Code Département','Département', 'Inscrits', 'Taux Abstention', 'Taux Blanc', 'Taux Nul', 'Taux Exprimé',
                  'MACRON', 'LE PEN']

In [ ]:
t2_dep.head()

### Données géographiques

In [ ]:
geo_dep = gpd.read_file('https://www.data.gouv.fr/fr/datasets/r/92f37c92-3aae-452c-8af1-c77e6dd590e5')

for i in geo_dep.index[96:]:
    dep = geo_dep.loc[i, 'libgeo']
    code = geo_dep.loc[i, 'dep']
    
    t1_dep.loc[t1_dep['Département'] == dep, 'Code Département'] = code
    t2_dep.loc[t2_dep['Département'] == dep, 'Code Département'] = code

In [ ]:
geo_dep.head()

## Résultats du premier tour

### Participation

### Répartition des votes

In [ ]:
# Premier tour
t1_rep_votes = t1_dep[t1_dep.columns[7:]].apply(sum).to_frame(name='Votes Obtenus')

t1_rep_votes.reset_index(inplace=True)
t1_rep_votes.rename(columns = {'index':'Candidats'}, inplace=True)
t1_rep_votes.sort_values(ascending=False, by='Votes Obtenus', inplace=True)

# Second tour
t2_rep_votes = t2_dep[t2_dep.columns[7:]].apply(sum).to_frame(name='Votes Obtenus')

t2_rep_votes.reset_index(inplace=True)
t2_rep_votes.rename(columns = {'index':'Candidats'}, inplace=True)
t2_rep_votes.sort_values(ascending=False, by='Votes Obtenus', inplace=True)

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    column_widths=[0.6, 0.4],
                    subplot_titles=['Premier tour', 'Second tour'],
                    y_title='Votes obtenus')

fig.add_trace(go.Bar(x=t1_rep_votes['Candidats'], y=t1_rep_votes['Votes Obtenus'],
                     text=t1_rep_votes['Votes Obtenus']),
             row=1, col=1)
fig.add_trace(go.Bar(x=t2_rep_votes['Candidats'], y=t2_rep_votes['Votes Obtenus'],
                     text=t2_rep_votes['Votes Obtenus']),
             row=1, col=2)

fig.update_layout(title_text='Élection Présidentielle 2022 - Répartition des votes',
                  showlegend=False)
fig.show()

## Analyse géographique

### Taux d'abstention

In [ ]:
fig_t1 = px.choropleth_mapbox(t1_dep,
                              locations='Code Département',
                              geojson=geo_dep,
                              color='Taux Abstention',
                              featureidkey='properties.dep',
                              hover_name='Département')
fig_t2 = px.choropleth_mapbox(t2_dep,
                              locations='Code Département',
                              geojson=geo_dep,
                              color='Taux Abstention',
                              featureidkey='properties.dep',
                              hover_name='Département')

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=['Premier tour', 'Second tour'],
                    specs=[[{'type': 'choroplethmapbox'}, {'type': 'choroplethmapbox'}]])

fig.add_trace(fig_t1['data'][0], row=1, col=1)
fig.add_trace(fig_t2['data'][0], row=1, col=2)

fig.update_mapboxes(style='white-bg',
                    center={'lat': 46.3, 'lon': 1.6751},
                    zoom=4.2)
fig.update_layout(title_text='Élection Présidentielle 2022 - Taux d\'abstention',
                  margin={'l': 0, 'r': 0, 't': 100, 'b': 0},
                  height=500,
                  coloraxis = {'colorscale': 'delta',
                               'cmin': 0,
                               'cmax': 100})

fig.show()

### Vainqueurs

In [ ]:
t1_vainq_dep = t1_dep[t1_dep.columns[7:]].idxmax(axis=1).to_frame(name='Vainqueur')
t1_vainq_dep['Code Département'] = t1_dep['Code Département']
t1_vainq_dep['Département'] = t1_dep['Département']

t2_vainq_dep = t2_dep[t2_dep.columns[7:]].idxmax(axis=1).to_frame(name='Vainqueur')
t2_vainq_dep['Code Département'] = t2_dep['Code Département']
t2_vainq_dep['Département'] = t2_dep['Département']

In [ ]:
fig = px.choropleth_mapbox(t2_vainq_dep,
                           locations='Code Département',
                           geojson=geo_dep,
                           color='Vainqueur',
                           featureidkey='properties.dep',
                           hover_name='Département',
                           mapbox_style='white-bg',
                           center={'lat': 46.3, 'lon': 1.6751},
                           zoom=4)
fig.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig.show()

In [ ]:
fig_t1 = px.choropleth_mapbox(t1_vainq_dep,
                              locations='Code Département',
                              geojson=geo_dep,
                              color='Vainqueur',
                              featureidkey='properties.dep',
                              hover_name='Vainqueur')
fig_t2 = px.choropleth_mapbox(t2_vainq_dep,
                              locations='Code Département',
                              geojson=geo_dep,
                              color='Vainqueur',
                              featureidkey='properties.dep',
                              hover_name='Vainqueur')

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=['Premier tour', 'Second tour'],
                    specs=[[{'type': 'choroplethmapbox'}, {'type': 'choroplethmapbox'}]])

fig.add_trace(fig_t1['data'][0], row=1, col=1)
fig.add_trace(fig_t2['data'][0], row=1, col=2)

fig.update_mapboxes(style='white-bg',
                    center={'lat': 46.3, 'lon': 1.6751},
                    zoom=4.2)
fig.update_layout(title_text='Élection Présidentielle 2022 - Vainqueur par département',
                  margin={'l': 0, 'r': 0, 't': 100, 'b': 0},
                  height=500)

fig.show()